In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
headers = {"Accept-Language": "en-US,en;q=0.5"}
url = 'https://www.popvortex.com/music/charts/top-100-songs.php'

response = requests.get(url,headers=headers)
response.status_code 
soup = BeautifulSoup(response.content, "html.parser")

In [4]:
#initialize empty lists
title = []
singers = []
genre = []


# define the number of iterations of our for loop
# by checking how many elements are in the retrieved result set
# (this is equivalent but more robust than just explicitly defining 250 iterations)
num_iter = len(soup.select("div.chart-wrapper div.chart-content"))

title_list = soup.select("div.chart-wrapper div.chart-content p.title-artist cite.title")
singer_list = soup.select("div.chart-wrapper div.chart-content p.title-artist em.artist")
genre_list = soup.select("div.chart-wrapper div.chart-content ul li a")

# iterate through the result set and retrive all the data
for i in range(num_iter):
    title.append(title_list[i].get_text())
    singers.append(singer_list[i].get_text().split('&'))
    genre.append(genre_list[i].get_text().split('/'))

    
# each list becomes a column
songs = pd.DataFrame({"title":title,
                       "singers":singers,
                       "genre":genre
                      })

songs

,title,singers,genre
0,Unholy,"[Sam Smith , Kim Petras]",[Pop]
1,Everywhere,[Fleetwood Mac],[Rock]
2,Eagle (feat. KB),[Transformation Worship],[New Release]
3,I'm Good (Blue),"[David Guetta , Bebe Rexha]","[Hip-Hop , Rap]"
4,A Thousand Years,[Christina Perri],[Dance]
...,...,...,...
95,We Don't Talk About Bruno,"[Carolina Gaitán - La Gaita, Mauro Castillo, A...",[Soundtrack]
96,Me Porto Bonito,"[Bad Bunny , Chencho Corleone]",[Country]
97,Whiskey Glasses,[Morgan Wallen],"[Hip-Hop , Rap]"
98,Rocky Mountain High,[John Denver],[Soundtrack]


In [5]:
def recommend_song(record):
    # first I check if the song is in the top 100
    if record in songs['title'].values:
        recommendations = pd.DataFrame()
        record_artists = songs[songs['title']==record]['singers']
        record_genres = songs[songs['title']==record]['genre']
        
        for artists in record_artists:
            for artist in artists:
                recommendations = pd.concat([recommendations,songs[songs['singers'].apply(lambda x: any([artist in singer for singer in x]))]],axis=0)

        for genres in record_genres:
            for genre in genres:
                print(genre)
                recommendations = pd.concat([recommendations,songs[songs['genre'].apply(lambda x: any([genre in genero for genero in x]))]],axis=0)               
                
        recommendations= recommendations.drop_duplicates(subset=['title'])

        return recommendations.iloc[np.random.randint(0,high=len(recommendations))]
        
    else:
        return "We don't have a recommendation for you."

In [6]:
recommend_song('Unholy')

Pop


title      Always Remember Us This Way
singers                    [Lady Gaga]
genre                            [Pop]
Name: 45, dtype: object